In [15]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import re
import os

import json
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException, NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

headers = {'User-Agent': 'Mozilla/5.0'}

In [7]:
url = "https://www.carmax.com/research"

driver = webdriver.Chrome(
    service=Service('/Users/iceyxue/Desktop/Data Design & Representation/HW/Final Project/chromedriver')
)
driver.get(url)

type_url_elements = driver.find_elements(By.CSS_SELECTOR, "div.researchByTypeTab_linkItemGroupContainer__rXSVD.researchByTypeTab_desktop__61B_4 > a")
type_urls = [element.get_attribute('href') for element in type_url_elements]

driver.quit()

print(type_urls)

['https://www.carmax.com/research/suvs', 'https://www.carmax.com/research/trucks', 'https://www.carmax.com/research/sedans', 'https://www.carmax.com/research/coupes', 'https://www.carmax.com/research/hatchbacks', 'https://www.carmax.com/research/convertibles', 'https://www.carmax.com/research/minivans-vans', 'https://www.carmax.com/research/station-wagons', 'https://www.carmax.com/research#Fuel%20type', 'https://www.carmax.com/research/electric-vehicles', 'https://www.carmax.com/research/plug-in-hybrids', 'https://www.carmax.com/research/hybrids', 'https://www.carmax.com/research/diesel-engines', 'https://www.carmax.com/research#Lifestyle', 'https://www.carmax.com/research/most-reliable-cars', 'https://www.carmax.com/research/sports-cars', 'https://www.carmax.com/research/luxury-vehicles', 'https://www.carmax.com/research/suvs/3-rows', 'https://www.carmax.com/research/best-awd-cars', 'https://www.carmax.com/research#Body%20type']


In [8]:
pd.DataFrame(type_urls).to_csv("type_urls.csv")

In [10]:
time.sleep(2)
driver = webdriver.Chrome(
    service=Service('/Users/iceyxue/Desktop/Data Design & Representation/HW/Final Project/chromedriver')
)
driver.get("https://www.carmax.com/research/suvs")

wait = WebDriverWait(driver, 20)

view_all_button_presense = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#VIEW-ALL-AVAILABLE-MODELS-info-section")))
driver.execute_script("arguments[0].scrollIntoView(true);", view_all_button_presense)

view_all_button_clickable = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#VIEW-ALL-AVAILABLE-MODELS-info-section")))
driver.execute_script("arguments[0].click();", view_all_button_clickable)

all_models_zone = driver.find_element(By.CSS_SELECTOR, "#VIEW-ALL-AVAILABLE-MODELS-description")
all_models_html = all_models_zone.get_attribute('outerHTML')
all_models_soup = BeautifulSoup(all_models_html, 'html.parser')
print(all_models_soup)

driver.quit()

<div class="kmx-disclosure-content" id="VIEW-ALL-AVAILABLE-MODELS-description"><div class="researchByMmyCategories_columns__dhf1_"><div><div class="researchByMmyCategories_card__eN0rb"><p class="researchByMmyCategories_title__8ad6A">Compact</p><ul class="researchByMmyCategories_categoryList__iTgOO"><li><a class="kmx-list-item-link" href="/research/buick/envision">Buick Envision</a></li><li><a class="kmx-list-item-link" href="/research/chevrolet/captiva-sport">Chevrolet Captiva Sport</a></li><li><a class="kmx-list-item-link" href="/research/chevrolet/equinox">Chevrolet Equinox</a></li><li><a class="kmx-list-item-link" href="/research/ford/bronco-sport">Ford Bronco Sport</a></li><li><a class="kmx-list-item-link" href="/research/ford/escape">Ford Escape</a></li><li><a class="kmx-list-item-link" href="/research/ford/escape-hybrid">Ford Escape Hybrid</a></li><li><a class="kmx-list-item-link" href="/research/ford/escape-plug-in-hybrid">Ford Escape Plug In Hybrid</a></li><li><a class="kmx-lis

In [13]:
print(all_models_soup.prettify())

<div class="kmx-disclosure-content" id="VIEW-ALL-AVAILABLE-MODELS-description">
 <div class="researchByMmyCategories_columns__dhf1_">
  <div>
   <div class="researchByMmyCategories_card__eN0rb">
    <p class="researchByMmyCategories_title__8ad6A">
     Compact
    </p>
    <ul class="researchByMmyCategories_categoryList__iTgOO">
     <li>
      <a class="kmx-list-item-link" href="/research/buick/envision">
       Buick Envision
      </a>
     </li>
     <li>
      <a class="kmx-list-item-link" href="/research/chevrolet/captiva-sport">
       Chevrolet Captiva Sport
      </a>
     </li>
     <li>
      <a class="kmx-list-item-link" href="/research/chevrolet/equinox">
       Chevrolet Equinox
      </a>
     </li>
     <li>
      <a class="kmx-list-item-link" href="/research/ford/bronco-sport">
       Ford Bronco Sport
      </a>
     </li>
     <li>
      <a class="kmx-list-item-link" href="/research/ford/escape">
       Ford Escape
      </a>
     </li>
     <li>
      <a class="kmx-

In [14]:
type_models_dict = {}

# Find all category cards
category_cards = all_models_soup.find_all("div", class_="researchByMmyCategories_card__eN0rb")

for card in category_cards:
    # The category name is in a <p> tag with a specific class
    category_name = card.find("p", class_="researchByMmyCategories_title__8ad6A").text.strip()
    # Find all <a> tags within <li> elements for the items in the category
    item_links = card.find_all("a", class_="kmx-list-item-link")
    # Extract the text from each item and create a list of items
    items = [link.text.strip() for link in item_links]
    # Assign the list of items to the category name in the dictionary
    type_models_dict[category_name] = items

type_models_dict

{'Compact': ['Buick Envision',
  'Chevrolet Captiva Sport',
  'Chevrolet Equinox',
  'Ford Bronco Sport',
  'Ford Escape',
  'Ford Escape Hybrid',
  'Ford Escape Plug In Hybrid',
  'Ford Mustang Mach-E (electric)',
  'GMC Terrain',
  'Honda Accord CrossTour',
  'Honda CR-V',
  'Honda CR-V Hybrid',
  'Hyundai Nexo',
  'Hyundai Santa Fe Sport',
  'Hyundai Tucson',
  'Hyundai Tucson Fuel Cell',
  'Hyundai Tucson Hybrid',
  'Hyundai Tucson PHEV',
  'Jeep Cherokee',
  'Jeep Compass',
  'Jeep Patriot',
  'Kia Niro',
  'Kia Sportage',
  'Kia Sportage Hybrid',
  'Kia Sportage PHEV',
  'Mazda CX-5',
  'Mazda CX-50',
  'Mazda CX-7',
  'Mazda MX-30 (electric)',
  'Nissan Ariya (electric)',
  'Nissan Rogue',
  'Nissan Rogue Select',
  'Subaru Forester',
  'Subaru Outback',
  'Subaru Solterra (electric)',
  'Toyota bZ4x (electric)',
  'Toyota Corolla Cross',
  'Toyota Corolla Cross Hybrid',
  'Toyota RAV4',
  'Toyota RAV4 EV',
  'Toyota RAV4 Hybrid',
  'Toyota RAV4 Prime Plug-In',
  'Toyota Venza',

In [17]:
type = "suv"
with open("source/type_models/{}.json".format(type), 'w') as f:
    json.dump(type_models_dict, f, indent=4)

In [18]:
driver = webdriver.Chrome(
    service=Service('/Users/iceyxue/Desktop/Data Design & Representation/HW/Final Project/chromedriver')
)
wait = WebDriverWait(driver, 20)

for url in type_urls:
    type_name = url.split("/")[-1][:-1]
    
    time.sleep(5)
    driver.get(url)

    # scroll the 'View All' button into view
    view_all_button_presense = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#VIEW-ALL-AVAILABLE-MODELS-info-section")))
    driver.execute_script("arguments[0].scrollIntoView(true);", view_all_button_presense)

    # click the 'View All' button to show full model list
    view_all_button_clickable = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#VIEW-ALL-AVAILABLE-MODELS-info-section")))
    driver.execute_script("arguments[0].click();", view_all_button_clickable)
    
    # locate the model list zone and read as soup
    all_models_zone = driver.find_element(By.CSS_SELECTOR, "#VIEW-ALL-AVAILABLE-MODELS-description")
    all_models_html = all_models_zone.get_attribute('outerHTML')
    all_models_soup = BeautifulSoup(all_models_html, 'html.parser')

    # store the sub-type and models in dict
    type_models_dict = {}

    # find all sub-type cards
    category_cards = all_models_soup.find_all("div", class_="researchByMmyCategories_card__eN0rb")

    for card in category_cards:
        # sub-type name is in a <p> tag with a specific class
        category_name = card.find("p", class_="researchByMmyCategories_title__8ad6A").text.strip()
        # find all <a> tags within <li> elements for the items in the sub-type
        item_links = card.find_all("a", class_="kmx-list-item-link")
        items = [link.text.strip() for link in item_links]
        # assign the list of models to the sub-type name in the dict
        type_models_dict[category_name] = items
    
    # save dict into json file
    with open("source/type_models/{}.json".format(type_name), 'w') as f:
        json.dump(type_models_dict, f, indent=4)

driver.quit()

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010057c53c chromedriver + 3966268
1   chromedriver                        0x0000000100574ac8 chromedriver + 3934920
2   chromedriver                        0x00000001001f7da0 chromedriver + 277920
3   chromedriver                        0x000000010023a394 chromedriver + 549780
4   chromedriver                        0x0000000100272bf0 chromedriver + 781296
5   chromedriver                        0x000000010022efb0 chromedriver + 503728
6   chromedriver                        0x000000010022fa28 chromedriver + 506408
7   chromedriver                        0x0000000100541724 chromedriver + 3725092
8   chromedriver                        0x0000000100545c18 chromedriver + 3742744
9   chromedriver                        0x000000010052a20c chromedriver + 3629580
10  chromedriver                        0x0000000100546714 chromedriver + 3745556
11  chromedriver                        0x000000010051d584 chro

In [ ]:
#__next > div.home_body__Vopgs > section.home_contentWidth__2TbV2.home_contentWidthMobile__Wm1z4 > div > div:nth-child(3) > div.researchByTypeTab_linkItemGroupContainer__rXSVD.researchByTypeTab_desktop__61B_4